In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = 'https://ucalgary.ca/pubs/calendar/current/en-4-1.html'

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [4]:
driver.get(url)
time.sleep(3)

# 1. Collect Course links for driver to click on

In [5]:
page_soup = soup(driver.page_source, 'lxml')

In [6]:
container = page_soup.find("span", {"id": "ctl00_ctl00_pageContent_ctl00_ctl02_cnBody"})

In [9]:
container

<span class="generic-body" id="ctl00_ctl00_pageContent_ctl00_ctl02_cnBody">
<p>The first-year curriculum is common to all programs except Energy Engineering.</p>
<ul>
<li><a class="link-text" href="engineering.html#30141">Engineering 200</a>, <a class="link-text" href="engineering.html#10159">201</a>, <a class="link-text" href="engineering.html#30142">202</a>, <a class="link-text" href="engineering.html#30143">225</a>
</li>
<li><a class="link-text" href="digital-engineering.html#47158">Digital Engineering 233</a>
</li>
<li><a class="link-text" href="mathematics.html#40096">Mathematics 275</a> or Applied Mathematics 217 
</li>
<li><a class="link-text" href="mathematics.html#40097">Mathematics 277</a> or Applied Mathematics 219 
</li>
<li><a class="link-text" href="mathematics.html#6942">Mathematics 211</a>
</li>
<li><a class="link-text" href="physics.html#6031">Physics 259</a>
</li>
<li><a class="link-text" href="chemistry.html#6509">Chemistry 209</a>
</li>
<li>General <a class="link-te

In [22]:
link_texts = container.findAll("a", {"class": "link-text"})
link_texts

[<a class="link-text" href="engineering.html#30141">Engineering 200</a>,
 <a class="link-text" href="engineering.html#10159">201</a>,
 <a class="link-text" href="engineering.html#30142">202</a>,
 <a class="link-text" href="engineering.html#30143">225</a>,
 <a class="link-text" href="digital-engineering.html#47158">Digital Engineering 233</a>,
 <a class="link-text" href="mathematics.html#40096">Mathematics 275</a>,
 <a class="link-text" href="mathematics.html#40097">Mathematics 277</a>,
 <a class="link-text" href="mathematics.html#6942">Mathematics 211</a>,
 <a class="link-text" href="physics.html#6031">Physics 259</a>,
 <a class="link-text" href="chemistry.html#6509">Chemistry 209</a>,
 <a class="link-text" href="en-3-3.html#3253">Complementary Studies </a>]

# 2. Test run - try to scrape the first course

In [21]:
pattern = re.compile("[A-Za-z ]*[0-9]{3}")

In [27]:
link = driver.find_element_by_partial_link_text(link_text[0].text)
link.click()

driver.current_url

'https://ucalgary.ca/pubs/calendar/current/engineering.html#30141'

 ### will need to extract the table id from the link to identify the target course's container

In [34]:
type(driver.current_url)

str

In [38]:
table_id = re.search("[0-9]{5}", driver.current_url)
table_id = table_id.group()
table_id

'30141'

In [39]:
page_soup = soup(driver.page_source, 'lxml')

In [43]:
container = page_soup.find("a", {"name": table_id})
container.parent

<div class="item-container"><a name="30141"></a>
<table align="center" bgcolor="white" border="0" bordercolor="#000000" cellpadding="2" cellspacing="0" width="100%">
<tbody><tr>
<td>
<table border="0" cellpadding="2" cellspacing="0" width="100%">
<tbody><tr>
<td bgcolor="#333333" valign="top" width="10%"><nobr>
<span class="course-code" id="ctl00_ctl00_pageContent_ctl03_ctl02_cnCourse">Engineering</span>
<span class="course-code" id="ctl00_ctl00_pageContent_ctl03_ctl02_cnCode">200</span>      </nobr>
</td>
<td bgcolor="#333333" valign="top" width="90%">
<span class="course-code" id="ctl00_ctl00_pageContent_ctl03_ctl02_cnTitle">Engineering Design, Innovation and Entrepreneurship</span>
</td>
</tr>
</tbody></table>
</td>
</tr>
<tr>
<td class="myCell">
<span class="course-desc" id="ctl00_ctl00_pageContent_ctl03_ctl02_cnDescription">Introduction to agile and iterative design; interdisciplinary application of engineering principles, digital technology, design, communications, leadership, en

In [48]:
container = container.parent

course_info = container.findAll("span", {"class": "course-code"})

course_code = course_info[0].text + " " + course_info[1].text
course_name = course_info[2].text

print(course_code)
print(course_name)

Engineering 200
Engineering Design, Innovation and Entrepreneurship


In [50]:
course_desc = container.find("span", {"class": "course-desc"}).text
course_desc

'Introduction to agile and iterative design; interdisciplinary application of engineering principles, digital technology, design, communications, leadership, entrepreneurship and project management concepts to a team-based design project with a focus on sustainability and social licences.'

In [51]:
driver.back()
driver.current_url

'https://ucalgary.ca/pubs/calendar/current/en-4-1.html'

In [67]:
driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-1.html")

link = driver.find_element_by_link_text(link_texts[1].text)
link.click()
page_soup = soup(driver.page_source, 'lxml')
table_id = driver.current_url[-5:]
container = page_soup.find("a", {"name": table_id})
driver.back()
driver.current_url

'https://ucalgary.ca/pubs/calendar/current/en-4-1.html'

# 3. Test run complete. Implement automation script to scrape all courses

In [69]:
from selenium.common.exceptions import NoSuchElementException

course_codes = []
course_names = []
course_descs = []
counter = 0

driver.get("https://ucalgary.ca/pubs/calendar/current/en-4-1.html")
    
for link_text in link_texts:
    if bool(re.match(pattern, link_text.text)):
        #go to course page
        try:
            link = driver.find_element_by_link_text(link_text.text)
        except NoSuchElementException:
            print("no link for {}".format(link_text.text))
            continue

        time.sleep(2)
        link.click()    
        time.sleep(2)
        
        #scrape course info
        page_soup = soup(driver.page_source, 'lxml')
        #get the id of the table containing the specific course information
        starting_index = driver.current_url.index("#")+1
        table_id = driver.current_url[starting_index:]
        
        #locate the container/table using the id
        container = page_soup.find("a", {"name": table_id})
        container = container.parent
        
        course_info = container.findAll("span", {"class": "course-code"})
        course_codes.append(course_info[0].text.strip() + " " + course_info[1].text.strip())
        course_names.append(course_info[2].text.strip())
        course_descs.append(container.find("span", {"class": "course-desc"}).text)
        
        print("Scraped ", course_codes[-1])
        counter += 1
        
        #go to course list page
        driver.back()
        time.sleep(2)
        
print("Finished scraping {} courses".format(counter))

Scraped  Engineering 200
Scraped  Engineering 201
Scraped  Engineering 202
Scraped  Engineering 225
Scraped  Digital Engineering 233
Scraped  Mathematics 275
Scraped  Mathematics 277
Scraped  Mathematics 211
Scraped  Physics 259
Scraped  Chemistry 209
Finished scraping 10 courses


# 4. Inspect and write to CSV

In [70]:
course_codes

['Engineering 200',
 'Engineering 201',
 'Engineering 202',
 'Engineering 225',
 'Digital Engineering 233',
 'Mathematics 275',
 'Mathematics 277',
 'Mathematics 211',
 'Physics 259',
 'Chemistry 209']

In [71]:
course_names

['Engineering Design, Innovation and Entrepreneurship',
 'Behaviour of Liquids, Gases and Solids',
 'Engineering Statics',
 'Fundamentals of Electrical Circuits and Machines',
 'Programming with Data',
 'Calculus for Engineers and Scientists',
 'Multivariable Calculus for Engineers and Scientists',
 'Linear Methods I',
 'Electricity and Magnetism (for students in Engineering)',
 'General Chemistry for Engineers']

In [72]:
course_descs

['Introduction to agile and iterative design; interdisciplinary application of engineering principles, digital technology, design, communications, leadership, entrepreneurship and project management concepts to a team-based design project with a focus on sustainability and social licences.',
 'An introduction to the behaviour of fluids and solids; phase transformations, the phase rule and phase diagrams. Ideal and real gases; equations of state and their engineering applications; simple kinetic theory; transport properties of fluids. Liquid state; vapor pressure; shear behaviour; flow of fluids in pipelines. Solids; crystalline and non-crystalline structure; non-equilibrium solid phases; electrical and thermal conductivity; dislocations; stress and strain; creep; fracture.',
 'Force vectors; equilibrium of a particle in two and three dimensions; force system resultants; equilibrium of a rigid body in two and three dimensions; internal forces in trusses; frames, machines and beams; bend

In [73]:
import pandas as pd

df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
    
})

df.to_csv('UCalgary_Engineering_Common_First_Year_Courses.csv', index = False)

In [ ]:
driver.quit()